In [3]:
import pathlib
import torch

import sentencepiece as spm
import recurrentgemma

import pprint as pp

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format="retina"

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
VARIANT = "2b-it"
weights_dir = kagglehub.model_download(f"google/recurrentgemma/pyTorch/{VARIANT}")

weights_dir = pathlib.Path(weights_dir)
ckpt_path = weights_dir/f"{VARIANT}.pt"
vocab_path = weights_dir/"tokenizer.model"

In [6]:
params = torch.load(str(ckpt_path))
params = {k: v.to(device=device) for k, v in params.items()}

In [7]:
model_config = recurrentgemma.GriffinConfig.from_torch_params(params)
model = recurrentgemma.Griffin(model_config, device=device, dtype=torch.bfloat16)
model.load_state_dict(params)

pp.pprint(model_config)

GriffinConfig(vocab_size=256000, width=2560, mlp_expanded_width=7680, num_heads=10, block_types=(<TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.ATTENTION: 1>, <TemporalBlockType.RECURRENT: 2>, <TemporalBlockType.RECURRENT: 2>), embeddings_scale_b

In [8]:
vocab = spm.SentencePieceProcessor()
vocab.Load(str(vocab_path))

True

In [9]:
sampler = recurrentgemma.Sampler(model=model, vocab=vocab)

In [19]:
input_batch = [
    "Robbert van Renesse is a professor of Computer Science at Cornell. Robbert",
]
out_data = sampler(input_strings=input_batch, total_generation_steps=30)

for input_string, out_string in zip(input_batch, out_data.text):
  print(f"prompt:\n{input_string}\noutput:\n{out_string}")
  print(10*'#')

TypeError: unsupported operand type(s) for +: 'Tensor' and 'NoneType'